In [3]:
#import Standard lib
from pathlib import PurePath, Path
import sys
import time
from collections import OrderedDict as od
import re
import os
import json

#import python scientific
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from numba import jit
import math
import pymc3 as pm
from theano import shared, theano as tt


# import visual tools
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
import seaborn as sns
import missingno as msno #결측치

plt.style.use('seaborn-talk')
plt.style.use('bmh')

# import util libs
import pyarrow as pa
import pyarrow.parquet as pq
from tqdm import tqdm, tqdm_notebook
import warnings
warnings.filterwarnings("ignore")

#my lib
from res import Analyze_bars, bar_function

In [7]:
def read_kibot_ticks(fp):
    # read tick data from http://www.kibot.com/support.aspx#data_format
    cols = list(map(str.lower,['Date','Time','Price','Bid','Ask','Size']))
    df = (pd.read_csv(fp, header=None)
          .rename(columns=dict(zip(range(len(cols)),cols)))
          .assign(dates=lambda df: (pd.to_datetime(df['date']+df['time'],
                                                  format='%m/%d/%Y%H:%M:%S')))
          .assign(v=lambda df: df['size']) # volume
          .assign(dv=lambda df: df['price']*df['size']) # dollar volume
          .drop(['date','time'],axis=1)
          .set_index('dates')
          .drop_duplicates())
    return df
    

df = read_kibot_ticks('C:/AFML/utills/IVE_tickbidask.txt')
print(df)

                      price     bid     ask  size     v         dv
dates                                                             
2009-09-28 09:30:00   50.79   50.70   50.79   100   100    5079.00
2009-09-28 09:30:00   50.71   50.70   50.79   638   638   32352.98
2009-09-28 09:31:32   50.75   50.75   50.76   100   100    5075.00
2009-09-28 09:31:33   50.75   50.72   50.75   100   100    5075.00
2009-09-28 09:31:50   50.75   50.73   50.76   300   300   15225.00
...                     ...     ...     ...   ...   ...        ...
2020-10-08 16:55:08  117.02  117.02  117.50   100   100   11702.00
2020-10-08 16:55:08  117.02  117.02  117.50  2428  2428  284124.56
2020-10-08 18:30:00  116.77  114.87  118.10     0     0       0.00
2020-10-08 19:53:14  117.50  115.06  117.50   100   100   11750.00
2020-10-08 20:00:00  116.77  115.06  117.50     0     0       0.00

[1958819 rows x 6 columns]
